# Process Election Results

In [1]:
import os
os.chdir('..')

from tqdm.notebook import tqdm
import pandas as pd

from scripts.data_transformations import (
    list_candidates
    , most_recent_smd
    )

from scripts.common import (
    hash_dataframe
    , match_names
)

In [2]:
results_files = {
    2020: 'November_3_2020_General_Election_Certified_Results.csv'
    , 2022: 'November_8_2022_General_Election_Certified_Results.csv'
}

for f in results_files:
    temp = pd.read_csv(f'data/dcboe/election_results/{results_files[f]}')
    

In [3]:
df = pd.read_csv(f'data/dcboe/election_results/{results_files[2022]}')
df = df.rename(columns={
    'Candidate': 'candidate_name'
    , 'ContestName': 'contest_name'
    , 'ContestNumber': 'contest_number'
    , 'Votes': 'votes'
})

In [4]:
# Topline race: Mayor
topline_results = (
    df[df['contest_name'] == 'MAYOR OF THE DISTRICT OF COLUMBIA DISTRICT OF COLUMBIA']
    .groupby('candidate_name')
    .votes.sum()
)

topline_results['total'] = topline_results.sum()
topline_results

candidate_name
DEM Muriel E. Bowser    147433
IND Rodney Red Grant     29531
LIB Dennis Sobin          2521
OVER VOTES                 241
REP Stacia R. Hall       11510
UNDER VOTES               7596
Write-in                  6580
total                   205412
Name: votes, dtype: int64

In [5]:
anc = df[df['contest_name'].str.contains('SINGLE MEMBER DISTRICT')].copy()

In [32]:
anc.votes.sum()

205412

In [7]:
# No certified results yet

# cert = pd.read_csv('../data/dcboe/November_3_2020_General_Election_Certified_Results.csv')
# cert = cert.rename(columns={
#     'Candidate': 'candidate_name'
#     , 'ContestName': 'contest_name'
#     , 'ContestNumber': 'contest_number'
#     , 'Votes': 'votes'
# })

# cert_anc = df[df['contest_name'].str.contains('SINGLE MEMBER DISTRICT')].copy()

# cert_anc.votes.sum()

In [8]:
anc.groupby('contest_name').votes.sum()

contest_name
ANC - 1A01 SINGLE MEMBER DISTRICT  01-ANC 1A     595
ANC - 1A02 SINGLE MEMBER DISTRICT  02-ANC 1A     521
ANC - 1A03 SINGLE MEMBER DISTRICT  03-ANC 1A     378
ANC - 1A04 SINGLE MEMBER DISTRICT  04-ANC 1A     641
ANC - 1A05 SINGLE MEMBER DISTRICT  05-ANC 1A     429
                                                ... 
ANC - 8F01 SINGLE MEMBER DISTRICT 01-ANC-8F      574
ANC - 8F02 SINGLE MEMBER DISTRICT 02-ANC-8F      986
ANC - 8F03 SINGLE MEMBER DISTRICT 03-ANC-8F      727
ANC - 8F04 SINGLE MEMBER DISTRICT 04-ANC-8F     1003
ANC - 8F05 SINGLE MEMBER DISTRICT 05-ANC-8F      757
Name: votes, Length: 345, dtype: int64

Did ANC vote counts change between Pre-Certified and Certified?

In [9]:
# todo
# sum(anc.groupby('contest_name').votes.sum() == cert_anc.groupby('contest_name').votes.sum())

In [10]:
anc['smd_id'] = 'smd_2022_' + anc['contest_name'].str.extract('(?<=ANC - )(.*)(?=SINGLE MEMBER)')
anc['smd_id'] = anc['smd_id'].str.strip()

In [11]:
# Check that smd_id parsed correctly
votes_by_anc = anc.groupby(['contest_name', 'smd_id']).votes.sum().reset_index()

districts = pd.read_csv('data/districts.csv')
districts = districts[districts.redistricting_year == 2022].copy()

votes_by_anc['valid'] = votes_by_anc['smd_id'].isin(districts.smd_id)

districts['in_results'] = districts['smd_id'].isin(votes_by_anc.smd_id)

# votes_by_anc.to_clipboard(index=False)

# There should be 345 valid districts
districts['in_results'].sum() == votes_by_anc['valid'].sum() == 345

True

In [12]:
candidates_results = (
    anc
    [~anc.candidate_name.isin(['OVER VOTES', 'UNDER VOTES'])]
    .groupby(['smd_id', 'contest_number', 'candidate_name'])
    .votes.sum()
    .reset_index()
)
# candidates

In [13]:
candidates_results['candidate_name_upper'] = candidates_results['candidate_name'].str.upper()
candidates_results['dcboe_hash_id'] = hash_dataframe(candidates_results, ['smd_id', 'candidate_name_upper'])
# candidates.loc[candidates.candidate_name == 'Write-in', 'dcboe_hash_id'] = pd.NA

In [14]:
candidates_results['ranking'] = candidates_results.groupby('smd_id').votes.rank(method='first', ascending=False)
candidates_results['winner'] = candidates_results['ranking'] == 1

In [15]:
candidates_results['write_in_winner'] = (
    candidates_results['winner'] & (candidates_results['candidate_name'] == 'Write-in')
)

In [16]:
# Sort candidates by the number of votes they got within SMD, making the winner first
candidates_results = candidates_results.sort_values(by=['smd_id', 'votes'], ascending=[True, False])

In [17]:
# Calculate the total votes cast in each SMD
total_votes = candidates_results.groupby('smd_id').votes.sum()
total_votes.name = 'total_votes'
candidates_results = pd.merge(candidates_results, total_votes, how='inner', on='smd_id')
candidates_results['vote_share'] = candidates_results['votes'] / candidates_results['total_votes']

In [18]:
# Create columns showing the number of votes the winner in each SMD received.
# This will be used to calculate the "margin of defeat" for all other candidates.

winning_votes = candidates_results.groupby('smd_id').agg(
    winning_votes=('votes', max)
    , winning_vote_percentage=('vote_share', max)
)

candidates_results = pd.merge(candidates_results, winning_votes, how='inner', on='smd_id')

In [19]:
# Create columns showing the number of votes for the next candidate in the DataFrame.
# For the first place candidate in an SMD, these _shifted columns will have the votes
# of the second-place candidate.

shift_one = candidates_results[['smd_id', 'votes', 'vote_share']].shift(-1)
shift_one = shift_one.rename(columns={
    'smd_id': 'smd_id_shifted'
    , 'votes': 'votes_shifted'
    , 'vote_share': 'vote_share_shifted'
})

candidates_results = pd.concat([candidates_results, shift_one], axis=1)

In [20]:
# Calculate the margin of victory - positive for winners, negative for losers
candidates_results['margin_of_victory'] = None
candidates_results['margin_of_victory_percentage'] = None

contested_winners = (
    (candidates_results['smd_id'] == candidates_results['smd_id_shifted']) & (candidates_results['winner'])
)

# For winners, the margin of victory is their votes minus the second-place votes
candidates_results.loc[contested_winners, 'margin_of_victory'] = (
    candidates_results['votes'] - candidates_results['votes_shifted']
)
candidates_results.loc[contested_winners, 'margin_of_victory_percentage'] = (
    candidates_results['vote_share'] - candidates_results['vote_share_shifted']
)

# For losers, the margin of defeat is the their votes minus the first-place votes
candidates_results.loc[~contested_winners, 'margin_of_victory'] = (
    candidates_results['votes'] - candidates_results['winning_votes']
)

candidates_results.loc[~contested_winners, 'margin_of_victory_percentage'] = (
    candidates_results['vote_share'] - candidates_results['winning_vote_percentage']
)

In [21]:
# Count the number of candidates who received votes. This lumps all write-ins as one candidate.
# num_candidates is not currently used by the frontend, just for humans checking the data.

num_candidates = candidates_results.groupby('smd_id').candidate_name.count()
num_candidates.name = 'num_candidates'
candidates_results = pd.merge(candidates_results, num_candidates, how='inner', on='smd_id')

## Counts

In [22]:
# Winners with the smallest margin of victory
(
    candidates_results
    [(candidates_results.winner) & (candidates_results.num_candidates > 1)]
    .sort_values(by='margin_of_victory_percentage')
    .head(10)
)

,smd_id,contest_number,candidate_name,votes,candidate_name_upper,dcboe_hash_id,ranking,winner,write_in_winner,total_votes,vote_share,winning_votes,winning_vote_percentage,smd_id_shifted,votes_shifted,vote_share_shifted,margin_of_victory,margin_of_victory_percentage,num_candidates
350,smd_2022_4E04,188,Carla Ferris,294,CARLA FERRIS,80845f8d7ce9bb471c8d9a6a6052efe3d0b4a1f7075910...,1.0,True,False,595,0.494118,294,0.494118,smd_2022_4E04,289.0,0.485714,5.0,0.008403,3
559,smd_2022_7C05,289,Shirley A. Boykins,137,SHIRLEY A. BOYKINS,1004c264e49d4f500f9800edef7c8462df6ba610027caf...,1.0,True,False,394,0.347716,137,0.347716,smd_2022_7C05,132.0,0.335025,5.0,0.01269,4
681,smd_2022_8D06,346,"Wendy ""Hope Dealer"" Hamilton",211,"WENDY ""HOPE DEALER"" HAMILTON",6331cdd90969e1238f6f2e60a13b21cbbf71a32ed628b4...,1.0,True,False,425,0.496471,211,0.496471,smd_2022_8D06,205.0,0.482353,6.0,0.014118,3
269,smd_2022_3F05,144,James Tandaric,258,JAMES TANDARIC,ed7b3f51826d49136959476509153714f23444c9ff6d91...,1.0,True,False,519,0.497110,258,0.497110,smd_2022_3F05,250.0,0.481696,8.0,0.015414,3
614,smd_2022_7F07,317,Shirley Thompson-Wright,178,SHIRLEY THOMPSON-WRIGHT,18826c65515fe7fd269c41df3b3b2f2f6615845703e91c...,1.0,True,False,471,0.377919,178,0.377919,smd_2022_7F07,170.0,0.360934,8.0,0.016985,4
276,smd_2022_4A01,153,Paula Y. Edwards,509,PAULA Y. EDWARDS,cb65a96724ad7b0e156c83f95df6e97f8542b83cd5a6d1...,1.0,True,False,1005,0.506468,509,0.506468,smd_2022_4A01,480.0,0.477612,29.0,0.028856,3
225,smd_2022_3C07,123,Gawain Kripke,349,GAWAIN KRIPKE,80eacd334c0a3c011ec57fae4ad46f4a50df16f3de1b9e...,1.0,True,False,681,0.512482,349,0.512482,smd_2022_3C07,320.0,0.469897,29.0,0.042584,3
416,smd_2022_5D06,219,Kathy Henderson,305,KATHY HENDERSON,9c114f4bf9e8b4dae95428034e012714ce70f41a1bf780...,1.0,True,False,586,0.520478,305,0.520478,smd_2022_5D06,276.0,0.470990,29.0,0.049488,3
219,smd_2022_3C05,121,Sauleh Ahmad Siddiqui,468,SAULEH AHMAD SIDDIQUI,258c83f1bb1a68cb87d470cb7d41c13004a4005fc764a9...,1.0,True,False,898,0.521158,468,0.521158,smd_2022_3C05,422.0,0.469933,46.0,0.051225,3
697,smd_2022_8E04,352,Kendall (The Voice) Simmons,105,KENDALL (THE VOICE) SIMMONS,8f890aab138068f0f90757878bb17aa48f52ca6360d258...,1.0,True,False,259,0.405405,105,0.405405,smd_2022_8E04,89.0,0.343629,16.0,0.061776,4


In [23]:
candidates_results['write_in_winner'].sum()

60

In [24]:
# candidates_results[candidates_results['write_in_winner']] #  & (candidates_results['num_candidates'] > 1)]

In [25]:
# Most votes
# candidates_results.sort_values(by='votes', ascending=False).head(10)

In [26]:
# candidates_results[candidates_results.smd_id == 'smd_2022_3F05']

## Match to OpenANC

Compare the candidates from DCBOE results to the OpenANC candidates table for matches

In [27]:
candidates_oa = list_candidates(election_year=2022)

In [28]:
merged = pd.merge(candidates_results, candidates_oa, how='left', on='dcboe_hash_id')

KeyError: 'dcboe_hash_id'

In [ ]:
merged.candidate_id.notnull().sum()

## District-wide stats

In [ ]:
anc.votes.sum()

In [ ]:
candidates_results.votes.sum()

In [ ]:
anc['candidate_type'] = anc.candidate_name
anc.loc[~anc.candidate_name.isin(['OVER VOTES', 'UNDER VOTES', 'Write-in']), 'candidate_type'] = 'Ballot Candidate'

In [ ]:
anc.groupby('candidate_type').votes.sum()

## Match to OpenANC database

In [ ]:
people = pd.read_csv('data/people.csv')
people = most_recent_smd(people)

In [ ]:
def match_to_openanc(df):

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        
        if row['candidate_name'] == 'Write-in':
            continue
        
        best_id, best_score = match_names(row['candidate_name'], people['full_name'], people['person_id'])

        df.loc[idx, 'match_score'] = best_score
        df.loc[idx, 'match_person_id'] = best_id
        df.loc[idx, 'match_full_name'] = people[people.person_id == best_id].full_name.iloc[0]
        df.loc[idx, 'match_smd_id'] = people[people.person_id == best_id].most_recent_smd_id.iloc[0]
        
    return df.copy()

In [ ]:
candidates_results = match_to_openanc(candidates_results)

In [ ]:
# candidates_results.to_clipboard()

In [ ]:
candidates_results

## Save output

In [ ]:
candidates_results['person_id'] = candidates_results['match_person_id']

candidates_results[[
    'person_id'
    , 'dcboe_hash_id'
    , 'smd_id'
    , 'candidate_name'
    , 'votes'
    , 'vote_share'
    , 'ranking'
    , 'winner'
    , 'write_in_winner'
    , 'margin_of_victory'
    , 'margin_of_victory_percentage'
    , 'num_candidates'
    , 'total_votes'
]].to_csv('data/dcboe/candidate_votes_2022.csv', index=False)

In [ ]:
# candidates_results.to_csv('candidates_results.csv', index=False)